In [ ]:
import os
import numpy as np
import xarray as xr
import pandas as pd
from netCDF4 import Dataset

def Srfdata_CoLM(srfdata_filedir, site, origfile_filepath):
    """
        生成单点的驱动数据
    """
    if len(origfile_filepath) > 0:
        #   1 读取原始文件
        srfdata = xr.open_dataset(origfile_filepath)
        #   2 修改原始文件中的部分信息
        srfdata["latitude"].values = np.array(site["lat"], dtype=np.float32)
        srfdata["longitude"].values = np.array(site["lon"], dtype=np.float32)
        srfdata["pfttyp"].values = np.array([15], dtype=np.int32)
        srfdata["pctpfts"].values = np.array([1.0], dtype=np.float32)
        #   3 保存数据
        srfdata_filepath = os.path.join(srfdata_filedir, "srfdata_{}_CoLM.nc".format(site["siteid"]))
        srfdata.to_netcdf(srfdata_filepath)
    else:
        #   1 生成输出文件
        srfdata_filepath = os.path.join(srfdata_filedir, f"{site["siteid"]}_{site["yrstt"]}-{site["yrend"]}_FLUXNET-CH4_Srf.nc")
        #   2 数据生成xarray
        srfdata = xr.Dataset({"latitude" : np.array(site["lat"], dtype=np.float32),
                            "longitude": np.array(site["lon"], dtype=np.float32),
                            "USGS_classification":3,
                            "IGBP_classification":np.array(site["igbptyp"], dtype=np.int32),
                            "pfttyp" : np.array(site["pfttyp"], dtype=np.int32),
                            "pctpfts": (["pft"], np.array([1.0], dtype=np.float32))})
        #   3 保存数据
        srfdata.to_netcdf(srfdata_filepath)


if __name__ == "__main__":
    #   1 基础信息
    srfdata_filedir = "/share/home/dq076/data/Srfdata/"
    if not os.path.exists(srfdata_filedir):
        os.makedirs(srfdata_filedir)

    data_dir = '/share/home/dq076/data/ME/FLUXNET-CH4/'
    df1 = pd.read_csv(os.path.join(data_dir, 'FLX_AA-Flx_CH4-META_20201112135337801132.csv'))    

    for i in range(df1.shape[0]):
        lat = df1.loc[i,'LAT']
        lon = df1.loc[i,'LON']
        siteid = df1.loc[i,'SITE_ID']
        pft = df1.loc[i,'SITE_CLASSIFICATION']
        if pft == 'Upland' or pft == 'Drained' and isinstance(df1.loc[i,'UPLAND_CLASS'], str):
            pft = df1.loc[i,'UPLAND_CLASS']
        pfttyp = {"Needleleaf forest": 1,"Mixed forest":1 ,"Tundra":12, "Alpine meadow": 13,"Grassland": 13,
                  "Crop - wheat":15, "Crop - corn":15, "Crop - alfalfa":15,"Crop - soy":15,
                  "Lake":79, "Rice":79,"Urban":0, "Drained":79,
                  "Fen":79, "Swamp": 79, "Bog": 79,
                  "Mangrove":79, "Wet tundra":79,
                  "Marsh":79, "Salt marsh":79}
        igbp = df1.loc[i,'IGBP']
        igbptyp = {"ENF": 1,"EBF":2 ,"DNF": 3,"DBF": 4,
                  "MF":5, "CSH":6, "OSH":7, "WSA":8,
                  "SAV":9,"GRA":10,"WET":11,"CRO":12,
                  "URB":13,"CNM":14,
                  "SNO":15, "BSV":16,"WAT":17,
                  }

        yrstt = df1.loc[i,'YEAR_START']
        yrend = df1.loc[i,'YEAR_END']
        origfile_filepath = ""

        print(siteid)
        site = {"siteid": siteid, "lat": lat, "lon": lon, "pfttyp": pfttyp[pft], 
                "yrstt": yrstt, "yrend":yrend, "igbptyp":igbptyp[igbp]}
        Srfdata_CoLM(srfdata_filedir, site, origfile_filepath)

AT-Neu
BR-Npw
BW-Gum
BW-Nxr
CA-SCB
CA-SCC
CH-Cha
CH-Dav
CH-Oe2
CN-Hgu
DE-Dgw
DE-Hte
DE-SfN
DE-Zrk
FI-Hyy
FI-Lom
FI-Si2
FI-Sii
FR-LGt
HK-MPM
ID-Pag
IT-BCi
IT-Cas
JP-BBY
JP-Mse
JP-SwL
KR-CRK
MY-MLM
NL-Hor
NZ-Kop
PH-RiF
RU-Ch2
RU-Che
RU-Cok
RU-Fy2
RU-Vrk
SE-Deg
SE-St1
UK-LBT
US-A03
US-A10
US-Atq
US-Beo
US-Bes
US-Bi1
US-Bi2
US-BZB
US-BZF
US-BZS
US-CRT
US-DPW
US-EDN
US-EML
US-Ho1
US-HRA
US-HRC
US-ICs
US-Ivo
US-LA1
US-LA2
US-Los
US-MAC
US-MRM
US-Myb
US-NC4
US-NGB
US-NGC
US-ORv
US-OWC
US-PFa
US-Snd
US-Sne
US-Srr
US-StJ
US-Tw1
US-Tw3
US-Tw4
US-Tw5
US-Twt
US-Uaf
US-WPT


In [5]:
import os
import numpy as np
import xarray as xr
import pandas as pd
from netCDF4 import Dataset
from joblib import Parallel, delayed

def process_site(i, df1, data_dir, forcing_dir):
    siteid = df1.loc[i, 'SITE_ID']
    print(siteid)
    yrstt = df1.loc[i, 'YEAR_START']
    yrend = df1.loc[i, 'YEAR_END']
    origfile_filepath = f'{data_dir}FLX_{siteid}_FLUXNET-CH4_{yrstt}-{yrend}_1-1/{siteid}_FLUXNET-CH4_HH.nc'
    print(origfile_filepath)
    data_path = f'{forcing_dir}{siteid}_{yrstt}-{yrend}_FLUXNET-CH4_Met.nc'
    os.system(f'cdo -setattribute,P_F_kgm2s@units="kg/m2/s" -aexpr,"P_F_kgm2s=P_F/1800" -setattribute,PA_F_Pa@units="Pa" -aexpr,"PA_F_Pa=PA_F*1000" -setattribute,TA_F_K@units="K" -aexpr,"TA_F_K=TA_F+273.15" -setattribute,q@units="kg/kg" -aexpr,"q=0.622*(es-VPD_F)/(PA_F*10-0.378*(es-VPD_F))" -aexpr,"es=6.1078*10^(7.5*TA_F/(TA_F+237.3))" -selname,TA_F,P_F,PA_F,WS_F,SW_IN_F,LW_IN_F,VPD_F,longitude,latitude {origfile_filepath} {data_path}')

if __name__ == "__main__":
    #   1 基础信息
    forcing_dir = "/share/home/dq076/data/Forcing/"
    if not os.path.exists(forcing_dir):
        os.makedirs(forcing_dir)

    data_dir = '/share/home/dq076/data/ME/FLUXNET-CH4/'
    df1 = pd.read_csv(os.path.join(data_dir, 'FLX_AA-Flx_CH4-META_20201112135337801132.csv'))
    df1 = df1[df1['FLUXNET-CH4_DATA_POLICY'] == 'CCBY4.0'].reset_index(drop=True)
    # 并行处理所有站点，使用所有可用CPU核心
    Parallel(n_jobs=-1)(delayed(process_site)(i, df1, data_dir, forcing_dir) for i in range(df1.shape[0]))

US-BZS
/share/home/dq076/data/ME/FLUXNET-CH4/FLX_US-BZS_FLUXNET-CH4_2015-2016_1-1/US-BZS_FLUXNET-CH4_HH.nc
MY-MLM
/share/home/dq076/data/ME/FLUXNET-CH4/FLX_MY-MLM_FLUXNET-CH4_2014-2015_1-1/MY-MLM_FLUXNET-CH4_HH.nc
US-Beo
/share/home/dq076/data/ME/FLUXNET-CH4/FLX_US-Beo_FLUXNET-CH4_2013-2014_1-1/US-Beo_FLUXNET-CH4_HH.nc
US-A03
ID-Pag
FI-Hyy
/share/home/dq076/data/ME/FLUXNET-CH4/FLX_US-A03_FLUXNET-CH4_2015-2018_1-1/US-A03_FLUXNET-CH4_HH.nc
/share/home/dq076/data/ME/FLUXNET-CH4/FLX_ID-Pag_FLUXNET-CH4_2016-2017_1-1/ID-Pag_FLUXNET-CH4_HH.nc
/share/home/dq076/data/ME/FLUXNET-CH4/FLX_FI-Hyy_FLUXNET-CH4_2016-2016_1-1/FI-Hyy_FLUXNET-CH4_HH.nc
IT-BCi
/share/home/dq076/data/ME/FLUXNET-CH4/FLX_IT-BCi_FLUXNET-CH4_2017-2018_1-1/IT-BCi_FLUXNET-CH4_HH.nc
DE-Hte
/share/home/dq076/data/ME/FLUXNET-CH4/FLX_DE-Hte_FLUXNET-CH4_2011-2018_1-1/DE-Hte_FLUXNET-CH4_HH.nc
JP-SwL
/share/home/dq076/data/ME/FLUXNET-CH4/FLX_JP-SwL_FLUXNET-CH4_2016-2016_1-1/JP-SwL_FLUXNET-CH4_HH.nc
BW-Nxr
/share/home/dq076/data/ME/FLUX